<a href="https://colab.research.google.com/github/cam2149/MachineLearningII/blob/main/Clustering-con-k-means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="markdown-google-sans">
  <h1>Ejemplo de clustering con k-means en Python</h1>
  Fuente: https://exponentis.es/ejemplo-de-clustering-con-k-means-en-python
  <br>
</div>

<div class="markdown-google-sans">
  Sin duda k-means es uno de los algoritmos de aprendizaje automático no supervisado más popular. El objetivo de k-means es simple: agrupa puntos de datos similares con el objetivo de descubrir patrones subyacentes. Para lograr este objetivo, k-means busca un número fijo (k) de agrupamientos (clústers) en el conjunto de datos .
  <h2>1. Funcionamiento básico de k-means</h2>
  Aunque estoy casi seguro que si has llegado hasta aquí ya conoces cómo funciona k-means (y lo que buscas es un ejemplo de uso) permíteme que te haga una muy breve descripción de su funcionamiento.

<p>Aunque estoy casi seguro que si has llegado hasta aquí ya conoces cómo funciona k-means (y lo que buscas es un ejemplo de uso) permíteme que te haga una muy breve descripción de su funcionamiento.</p>
<p>En k-means<strong> se define de inicio un número </strong><em><strong>k</strong></em>, que se refiere al número de centroides en los que se dividirá el conjunto de datos. Cada centroide sería la ubicación que marca el centro de cada agrupación.<br><strong>A cada punto se asigna uno de los grupos mediante la reducción de la suma de cuadrados en el grupo</strong>. Dicho de otra forma, el algoritmo k-means identifica <em>k</em> número de centroides, y luego asigna cada punto de los datos al grupo más cercano, mientras mantiene los centroides lo más pequeños posible.</p>
<p>Una vez tenemos cada punto asociado a un clúster, <strong>podemos etiquetarlo en el <em>dataframe</em> original</strong> asociándolo a dicho grupo y “catalogando” por tanto nuestros datos.</p>
</div>

<h4 class="wp-block-heading">2. Datos de inicio: valores de las acciones de Samsung.</h4>
<p>Vamos a aplicar el algoritmo sobre <strong>un conjunto de datos de las acciones de </strong><a href="https://markets.businessinsider.com/stocks/samsung-stock"><strong>Samsung</strong></a><strong> en bolsa</strong> para determinar cómo se agrupan usando k-means. Para ello nos descargamos de la web de <a rel="noreferrer noopener" aria-label="Yahoo Finanzas (abre en una nueva pestaña)" href="https://es.finance.yahoo.com/quote/005930.KS?p=005930.KS&amp;.tsrc=fin-srch" target="_blank">Yahoo Finanzas</a> el histórico con los valores desde el 1 de enero de 2008 de las acciones de <em>Samsung Electronics Co., Ltd</em>.</p>
<p>De los datos descargados, buscaremos <strong>patrones de agrupamiento entre dos señales elegidas</strong>, en este caso seleccionamos la señal <strong>CLOSE</strong> (precio de cierre diario) y la señal <strong>VOLUME</strong> (volumen de contrataciones diario). Contaremos con una <em>dataframe</em> con los <strong>datos diarios desde el 1 de enero de 2008 hasta el 28 de junio de 2019</strong>, lo que nos da un total de 2849 registros (que serán algo menos cuando limpiemos los valores nulos).</p>
<p>NOTA: podemos aplicar k-means sobre todas las señales/variables necesarias, pero para poder visualizar los agrupamientos k-means en una gráfica 2-D aplicaremos el algoritmo exclusivamente sobre la dos señales anteriormente detalladas.</p>



<h4 class="wp-block-heading">3. Carga de librerías y del <em>dataframe</em>.</h4>

<p>Las librerías usadas serán <a rel="noreferrer noopener" aria-label="Pandas (abre en una nueva pestaña)" href="https://pandas.pydata.org/" target="_blank">Pandas</a> para almacenar y manipular el <em>dataframe</em>, <a rel="noreferrer noopener" href="https://scikit-learn.org/stable/modules/preprocessing.html" target="_blank">Preprocessing</a> para normalizar los datos antes de aplicar al algoritmo, <a rel="noreferrer noopener" aria-label="KMeans (abre en una nueva pestaña)" href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html" target="_blank">KMeans</a> para realizar el clustering,  y <a href="https://matplotlib.org/3.1.1/tutorials/introductory/pyplot.html">Pyplot</a> para hacer las representaciones gráficas.</p>

<p>El <em>dataframe</em> cargado queda de la siguiente forma:</p>

In [2]:
#%% Carga del dataframe.
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [13]:
#%% Carga del dataframe.
url_data = 'https://github.com/cam2149/MachineLearningII/raw/08967e0c9145c90135fec839db6f24ca41ab7937/Samsung_11_14_24-01_02_08.csv'
new_headers = ['Open', 'Close', 'DailyHigh', 'DaiLow', 'High', 'Low','Volume','Date']
dfSamsung = pd.read_csv(url_data, header=0, names=new_headers)

dfSamsung['High'] = dfSamsung['High'].astype(str)
dfSamsung['Low'] = dfSamsung['Low'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].str.replace('.0', '')
dfSamsung['Volume'] = dfSamsung['High'] + dfSamsung['Low'] + dfSamsung['Volume']
dfSamsung = dfSamsung.drop(['High', 'Low'], axis=1)
dfSamsung[0:5]


,Open,Close,DailyHigh,DaiLow,Volume,Date
0,50200,49900,51800,49900,45107400,11/14/24
1,52000,50600,53000,50500,50647543,11/13/24
2,54600,53000,54600,53000,32714953,11/12/24
3,56700,55000,56800,55000,29468632,11/11/24
4,58000,57000,58300,57000,13432473,11/08/24


In [5]:
# prompt: Eliminar 'High', 'Low',

import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# <div class="markdown-google-sans">
#   <h1>Ejemplo de clustering con k-means en Python</h1>
#   Fuente: https://exponentis.es/ejemplo-de-clustering-con-k-means-en-python
#   <br>
# </div>
#
# <div class="markdown-google-sans">
#   Sin duda k-means es uno de los algoritmos de aprendizaje automático no supervisado más popular. El objetivo de k-means es simple: agrupa puntos de datos similares con el objetivo de descubrir patrones subyacentes. Para lograr este objetivo, k-means busca un número fijo (k) de agrupamientos (clústers) en el conjunto de datos .
#   <h2>1. Funcionamiento básico de k-means</h2>
#   Aunque estoy casi seguro que si has llegado hasta aquí ya conoces cómo funciona k-means (y lo que buscas es un ejemplo de uso) permíteme que te haga una muy breve descripción de su funcionamiento.
#
# <p>Aunque estoy casi seguro que si has llegado hasta aquí ya conoces cómo funciona k-means (y lo que buscas es un ejemplo de uso) permíteme que te haga una muy breve descripción de su funcionamiento.</p>
# <p>En k-means<strong> se define de inicio un número </strong><em><strong>k</strong></em>, que se refiere al número de centroides en los que se dividirá el conjunto de datos. Cada centroide sería la ubicación que marca el centro de cada agrupación.<br><strong>A cada punto se asigna uno de los grupos mediante la reducción de la suma de cuadrados en el grupo</strong>. Dicho de otra forma, el algoritmo k-means identifica <em>k</em> número de centroides, y luego asigna cada punto de los datos al grupo más cercano, mientras mantiene los centroides lo más pequeños posible.</p>
# <p>Una vez tenemos cada punto asociado a un clúster, <strong>podemos etiquetarlo en el <em>dataframe</em> original</strong> asociándolo a dicho grupo y “catalogando” por tanto nuestros datos.</p>
# </div>
#
# <h4 class="wp-block-heading">2. Datos de inicio: valores de las acciones de Samsung.</h4>
# <p>Vamos a aplicar el algoritmo sobre <strong>un conjunto de datos de las acciones de </strong><a href="https://markets.businessinsider.com/stocks/samsung-stock"><strong>Samsung</strong></a><strong> en bolsa</strong> para determinar cómo se agrupan usando k-means. Para ello nos descargamos de la web de <a rel="noreferrer noopener" aria-label="Yahoo Finanzas (abre en una nueva pestaña)" href="https://es.finance.yahoo.com/quote/005930.KS?p=005930.KS&amp;.tsrc=fin-srch" target="_blank">Yahoo Finanzas</a> el histórico con los valores desde el 1 de enero de 2008 de las acciones de <em>Samsung Electronics Co., Ltd</em>.</p>
# <p>De los datos descargados, buscaremos <strong>patrones de agrupamiento entre dos señales elegidas</strong>, en este caso seleccionamos la señal <strong>CLOSE</strong> (precio de cierre diario) y la señal <strong>VOLUME</strong> (volumen de contrataciones diario). Contaremos con una <em>dataframe</em> con los <strong>datos diarios desde el 1 de enero de 2008 hasta el 28 de junio de 2019</strong>, lo que nos da un total de 2849 registros (que serán algo menos cuando limpiemos los valores nulos).</p>
# <p>NOTA: podemos aplicar k-means sobre todas las señales/variables necesarias, pero para poder visualizar los agrupamientos k-means en una gráfica 2-D aplicaremos el algoritmo exclusivamente sobre la dos señales anteriormente detalladas.</p>
#
#
# <h4 class="wp-block-heading">3. Carga de librerías y del <em>dataframe</em>.</h4>
# <p>Las librerías usadas serán <a rel="noreferrer noopener" aria-label="Pandas (abre en una nueva pestaña)" href="https://pandas.pydata.org/" target="_blank">Pandas</a> para almacenar y manipular el <em>dataframe</em>, <a rel="noreferrer noopener" href="https://scikit-learn.org/stable/modules/preprocessing.html" target="_blank">Preprocessing</a> para normalizar los datos antes de aplicar al algoritmo, <a rel="noreferrer noopener" aria-label="KMeans (abre en una nueva pestaña)" href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html" target="_blank">KMeans</a> para realizar el clustering,  y <a href="https://matplotlib.org/3.1.1/tutorials/introductory/pyplot.html">Pyplot</a> para hacer las representaciones gráficas.</p>
#
# <p>El <em>dataframe</em> cargado queda de la siguiente forma:</p>
#%% Carga del dataframe.
#%% Carga del dataframe.
url_data = 'https://github.com/cam2149/MachineLearningII/raw/08967e0c9145c90135fec839db6f24ca41ab7937/Samsung_11_14_24-01_02_08.csv'
new_headers = ['Open', 'Close', 'DailyHigh', 'DaiLow', 'High', 'Low','Volume','Date']
dfSamsung = pd.read_csv(url_data, header=0, names=new_headers)

dfSamsung['High'] = dfSamsung['High'].astype(str)
dfSamsung['Low'] = dfSamsung['Low'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].str.replace('.0', '')
# Eliminar las columnas 'High' y 'Low'
dfSamsung = dfSamsung.drop(['High', 'Low'], axis=1)
dfSamsung['Volume'] = dfSamsung['DailyHigh'] + dfSamsung['DaiLow'] + dfSamsung['Volume']
dfSamsung[0:5]

,,Open,Close,High,Low,Volume,Date
50200,49900,51800,49900,45,107,400.0,11/14/24
52000,50600,53000,50500,50,647,543.0,11/13/24
54600,53000,54600,53000,32,714,953.0,11/12/24
56700,55000,56800,55000,29,468,632.0,11/11/24
58000,57000,58300,57000,13,432,473.0,11/08/24


In [ ]:
dfSamsung.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4164 entries, (50200, 49900) to (11180, 10880)
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    4164 non-null   int64  
 1   Close   4164 non-null   int64  
 2   High    4164 non-null   int64  
 3   Low     4164 non-null   int64  
 4   Volume  4164 non-null   float64
 5   Date    4164 non-null   object 
dtypes: float64(1), int64(4), object(1)
memory usage: 366.3+ KB


In [ ]:
# Convertir las columnas 'High', 'Low', 'Volume' a string
dfSamsung['DailyHigh'] = dfSamsung['DailyHigh'].astype(str)
dfSamsung['DaiLow'] = dfSamsung['DaiLow'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].astype(str)
dfSamsung['Volume'] = dfSamsung['Volume'].str.replace('.0', '')

KeyError: 'DailyHigh'

In [ ]:
# prompt: Convierte las columnas Volume	 en int

# ... (Your existing code)

# Eliminar comas y convertir a entero la columna 'Volume'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4164 non-null   int64  
 1   Close      4164 non-null   int64  
 2   DailyHigh  4164 non-null   int64  
 3   DailyLow   4164 non-null   int64  
 4   High       4164 non-null   int64  
 5   Low        4164 non-null   int64  
 6   Volume     4164 non-null   float64
 7   Date       4164 non-null   object 
dtypes: float64(1), int64(6), object(1)
memory usage: 260.4+ KB


ValueError: invalid literal for int() with base 10: '45107400.0'

<h4 class="wp-block-heading">4. Preprocesado de los datos.</h4>
<p>En primer lugar, eliminamos los <a rel="noreferrer noopener" aria-label="datos vacíos (NA)  (abre en una nueva pestaña)" href="https://es.wikipedia.org/wiki/N/a" target="_blank">datos vacíos (NA) </a>y resetamos el índice. El objetivo de dicho reseteo es que el índice que numera las filas no se reajusta cuando eliminamos filas vacías, y pueden presentarse problemas de dimensionalidad al extraer columnas para posteriormente agregarlas:</p>

In [ ]:
 #%% Se eliminan filas que tengan valor NaN.
 dfSamsung = dfSamsung.dropna()
 dfSamsung = dfSamsung.reset_index(drop=True)

In [ ]:
dfSamsung.describe()

,Open,Close,DailyHigh,DailyLow
count,4164.000000,4164.000000,4164.000000,4164.000000
mean,39306.515370,39285.857349,39678.244476,38914.209894
std,21930.644801,21899.503515,22099.435551,21738.436827
min,8150.000000,8150.000000,8630.000000,8060.000000
25%,22000.000000,22100.000000,22330.000000,21800.000000
50%,30250.000000,30320.000000,30590.000000,29960.000000
75%,56900.000000,56770.000000,57500.000000,56100.000000
max,90300.000000,91000.000000,96800.000000,89500.000000


<p>En nuestro ejemplo el <em>dataset</em> “limpio” es de 2784 registros, es decir, que se han eliminado 66 registros NA.</p>